In [4]:
!pip install yfinance
!pip install ta
!pip install mpl_finance
!pip install mplfinance

In [5]:
import pandas as pd
import statistics
import numpy as np
import yfinance
import ta
from scipy.signal import argrelextrema
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import mplfinance as mpf
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]
plt.rc('font', size=14)

In [6]:
name = ['ADANIPORTS.NS', 'ASIANTILES.NS', 'BANDHANBNK.NS', 'BHEL.NS', 'BODALCHEM.NS', 'BRIGADE.NS', 'ZENSARTECH.NS', 'WELSPUNIND.NS', 'VOLTAS.NS', 'MANYAVAR.NS', 'TATASTEEL.NS', 'SOBHA.NS', 'SBICARD.NS', 'SAPPHIRE.NS', 'SAGCEM.NS', 'ROUTE.NS', 'RITES.NS', 'RBA.NS', 'RCOM.NS', 'RBLBANK.NS', 'RANASUG.NS', 'PRESTIGE.NS', 'PNCINFRA.NS', 'OFSS.NS', 'PAYTM.NS', 'NSLNISP.NS', 'NMDC.NS', 'MTARTECH.NS', 'MON100.NS', 'MAXHEALTH.NS', 'LTIM.NS', 'LICI.NS', 'KPIGREEN.NS', 'KMSUGAR.NS', 'JAMNAAUTO.NS', 'JISLJALEQS.NS', 'IPCALAB.NS', 'IOLCP.NS', 'IOC.NS', 'HFCL.NS', 'HDFCNIFETF.NS', 'HDFCLIFE.NS', 'HCLTECH.NS', 'GREAVESCOT.NS', 'GMMPFAUDLR.NS', 'GATEWAY.NS', 'NYKAA.NS', 'EMBASSY.BO', 'EIHOTEL.NS', 'DELTACORP.NS', 'DEEPAKNTR.NS', 'CDSL.NS', 'CAPLIPOINT.NS']
ticker = list(map(lambda x: yfinance.Ticker(x), name))
df = {}
df2= {}
for i,j in zip(ticker, name):
  dataframe = i.history(interval="1d",start="2023-01-02", end="2023-07-07")
  dataframe2 = dataframe.copy()
  dataframe2.reset_index(inplace=True)
  dataframe2['date'] = pd.to_datetime(dataframe2['Date'])
  dataframe2 = dataframe2.set_index('date')
  dataframe2 = dataframe2.loc[:,['Open', 'High', 'Low', 'Close']]
  dataframe2.columns=  ['open', 'high', 'low','close']
  dataframe['Date'] = pd.to_datetime(dataframe.index)
  dataframe['Date'] = dataframe['Date'].apply(mpl_dates.date2num)
  dataframe = dataframe.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]
  df[j] = dataframe
  df2[j] =dataframe2

## Fractal Method for S/R

In [7]:
def isSupport(df,i):
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]
  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2]
  return resistance


def isFarFromLevel(l):
   return np.sum([abs(l-x) < s  for x in levels]) == 0


def using_extremas(points_to_consider, df):
  price_data_low = df['Low'].values
  price_data_high = df['High'].values
  local_minima_indices = argrelextrema(price_data_low, np.less, order = points_to_consider)[0]
  local_maxima_indices = argrelextrema(price_data_high, np.greater , order = points_to_consider)[0]

  return local_minima_indices, local_maxima_indices


def using_gap(threshold, df):
  gap_up_index = []
  gap_down_index = []
  for i in range(1, len(df)):
    if (df['Low'][i] - df['High'][i-1]) > 0:
      if ((df['Low'][i] - df['High'][i-1]) / df['High'][i-1] ) * 100 > threshold:
        gap_up_index.append(i)
    elif (df['High'][i] - df['Low'][i-1]) <0 :
      if ((df['Low'][i-1] - df['High'][i]) / df['Low'][i-1] ) * 100 > threshold:
        gap_down_index.append(i)
  return gap_up_index, gap_down_index


def find_mode(lst):
    try:
        mode = statistics.mode(lst)
        return mode
    except statistics.StatisticsError:
        return "No unique mode found"

In [8]:
local_minima_indices={}
local_maxima_indices = {}
for i in name:
  data_for_extrema = df[i]
  local_minima_indices[i], local_maxima_indices[i] = using_extremas(20,data_for_extrema)


In [9]:
# plt.style.use('fivethirtyeight')
# plt.rcParams['figure.figsize'] = (20, 10)
# def get_ci(high, low, close, lookback):
#     tr1 = pd.DataFrame(high - low).rename(columns = {0:'tr1'})
#     tr2 = pd.DataFrame(abs(high - close.shift(1))).rename(columns = {0:'tr2'})
#     tr3 = pd.DataFrame(abs(low - close.shift(1))).rename(columns = {0:'tr3'})
#     frames = [tr1, tr2, tr3]
#     tr = pd.concat(frames, axis = 1, join = 'inner').dropna().max(axis = 1)
#     atr = tr.rolling(1).mean()
#     highh = high.rolling(lookback).max()
#     lowl = low.rolling(lookback).min()
#     ci = 100 * np.log10((atr.rolling(lookback).sum()) / (highh - lowl)) / np.log10(lookback)
#     return ci

# df2['ci_14'] = get_ci(df2['high'], df2['low'], df2['close'], 10)

# ax1 = plt.subplot2grid((11,1,), (0,0), rowspan = 5, colspan = 1)
# ax2 = plt.subplot2grid((11,1,), (6,0), rowspan = 4, colspan = 1)
# ax1.plot(df2['close'], linewidth = 2.5, color = '#2196f3')
# ax1.set_title('TSLA CLOSING PRICES')
# ax2.plot(df2['ci_14'], linewidth = 2.5, color = '#fb8c00')
# ax2.axhline(38.2, linestyle = '--', linewidth = 1.5, color = 'grey')
# ax2.axhline(61.8, linestyle = '--', linewidth = 1.5, color = 'grey')
# ax2.set_title('TSLA CHOPPINESS INDEX 14')
# plt.show()

In [10]:
opening_gap_up_index = {}
opening_gap_down_index = {}
for i in name:
  data_for_gaps = df[i]
  opening_gap_up_index[i] , opening_gap_down_index[i] = using_gap(0.5, data_for_gaps)

In [11]:
all_levels = {}
for j in name:
  levels = []
  data_for_levels = df[j]
  s =  np.mean(data_for_levels['High'] - data_for_levels['Low'])
  for i in range(2, data_for_levels.shape[0] - 2):
    if isSupport(data_for_levels, i):
        l = data_for_levels['Low'][i]
        if isFarFromLevel(l):
            upper = l + 0.01 * l
            lower = l - 0.01 * l
            levels.append([i, l, upper, lower])
    elif isResistance(data_for_levels, i):
        l = data_for_levels['High'][i]
        if isFarFromLevel(l):
            upper = l + 0.01 * l
            lower = l - 0.01 * l
            levels.append([i, l, upper, lower])
    elif i in local_minima_indices:
        l = data_for_levels['Low'][i]
        if isFarFromLevel(l):
            upper = l + 0.01 * l
            lower = l - 0.01 * l
            levels.append([i, l, upper, lower])
    elif i in local_maxima_indices:
        l = data_for_levels['High'][i]
        if isFarFromLevel(l):
            upper = l + 0.01 * l
            lower = l - 0.01 * l
            levels.append([i, l, upper, lower])
    elif i in opening_gap_up_index:
        l = data_for_levels['Low'][i]
        o = data_for_levels['High'][i - 1]
        upper = l + 0.01 * l
        lower = l - 0.01 * l
        levels.append([i, l, upper, lower])
        upper2 = o + 0.01 * o
        lower2 = o - 0.01 * o
        levels.append([i - 1, o, upper2, lower2])
    elif i in opening_gap_down_index:
        l = data_for_levels['High'][i]
        o = data_for_levels['Low'][i - 1]
        upper = l + 0.01 * l
        lower = l - 0.01 * l
        levels.append([i, l, upper, lower])
        upper2 = o + 0.01 * o
        lower2 = o - 0.01 * o
        levels.append([i - 1, o, upper2, lower2])

    all_levels[j] = levels


In [12]:
sr_price_diff_percs = {}
for i in name:
  price_diff = {}
  for j in range(len(all_levels[i])):
    close_price = df[i].iloc[-1]['Close']
    min_price_diff = (abs(close_price - all_levels[i][j][1]) / all_levels[i][j][1]) *100
    price_diff[all_levels[i][j][0]] = min_price_diff
  sr_price_diff_percs[i] = price_diff



In [13]:
perfect_sr_price_diff_perc = {}
for i in name:
  value = min(list(sr_price_diff_percs[i].values()))
  perfect_sr_price_diff_perc[i] = value


## Trendlines



In [14]:
def check_trend_line(support: bool, pivot: int, slope: float, y: np.array):
    # compute sum of differences between line and prices,
    # return negative val if invalid

    # Find the intercept of the line going through pivot point with given slope
    intercept = -slope * pivot + y[pivot]
    line_vals = slope * np.arange(len(y)) + intercept

    diffs = line_vals - y

    # Check to see if the line is valid, return -1 if it is not valid.
    if support and diffs.max() > 1e-5:
        return -1.0
    elif not support and diffs.min() < -1e-5:
        return -1.0

    # Squared sum of diffs between data and line
    err = (diffs ** 2.0).sum()
    return err;


def optimize_slope(support: bool, pivot:int , init_slope: float, y: np.array):

    # Amount to change slope by. Multiplyed by opt_step
    slope_unit = (y.max() - y.min()) / len(y)

    # Optmization variables
    opt_step = 1.0
    min_step = 0.0001
    curr_step = opt_step # current step

    # Initiate at the slope of the line of best fit
    best_slope = init_slope
    best_err = check_trend_line(support, pivot, init_slope, y)
    assert(best_err >= 0.0) # Shouldn't ever fail with initial slope

    get_derivative = True
    derivative = None
    while curr_step > min_step:

        if get_derivative:
            # Numerical differentiation, increase slope by very small amount
            # to see if error increases/decreases.
            # Gives us the direction to change slope.
            slope_change = best_slope + slope_unit * min_step
            test_err = check_trend_line(support, pivot, slope_change, y)
            derivative = test_err - best_err;

            # If increasing by a small amount fails,
            # try decreasing by a small amount
            if test_err < 0.0:
                slope_change = best_slope - slope_unit * min_step
                test_err = check_trend_line(support, pivot, slope_change, y)
                derivative = best_err - test_err

            if test_err < 0.0: # Derivative failed, give up
                raise Exception("Derivative failed. Check your data. ")

            get_derivative = False

        if derivative > 0.0: # Increasing slope increased error
            test_slope = best_slope - slope_unit * curr_step
        else: # Increasing slope decreased error
            test_slope = best_slope + slope_unit * curr_step


        test_err = check_trend_line(support, pivot, test_slope, y)
        if test_err < 0 or test_err >= best_err:
            # slope failed/didn't reduce error
            curr_step *= 0.5 # Reduce step size
        else: # test slope reduced error
            best_err = test_err
            best_slope = test_slope
            get_derivative = True # Recompute derivative

    # Optimize done, return best slope and intercept
    return (best_slope, -best_slope * pivot + y[pivot])



def fit_trendlines_high_low(high: np.array, low: np.array, close: np.array):
    x = np.arange(len(close))
    coefs = np.polyfit(x, close, 1)
    # coefs[0] = slope,  coefs[1] = intercept
    line_points = coefs[0] * x + coefs[1]
    upper_pivot = (high - line_points).argmax()
    lower_pivot = (low - line_points).argmin()

    support_coefs = optimize_slope(True, lower_pivot, coefs[0], low)
    resist_coefs = optimize_slope(False, upper_pivot, coefs[0], high)

    return (support_coefs, resist_coefs)




In [15]:
trendline_price_diff_percs = {}
for i in name:
  price_diff ={}
  data = df2[i].copy()
  for j in [-30, -40, -50, -60, -70, -80, -90, -100]:
    for k in [-25, -20, -15, -10, -5]:
      candles = data.iloc[j:k]
      support_coefs, resist_coefs = fit_trendlines_high_low(candles['high'], candles['low'], candles['close'])
      close_price = data.iloc[-1]['close']
      support_trendline_price = support_coefs[0] * (abs(j)) + support_coefs[1]
      resistance_trendline_price = resist_coefs[0] * (abs(j)) + resist_coefs[1]
      min_price_diff = min((abs(close_price - support_trendline_price) / support_trendline_price) *100, (abs(close_price - resistance_trendline_price) / resistance_trendline_price) *100)
      level = (j,k)
      price_diff[level] = min_price_diff
  trendline_price_diff_percs[i] = price_diff


In [16]:
trendline_threshold_filter = 3

perfect_trendline_price_diff_perc = {}
for i in name:
  values = list(trendline_price_diff_percs[i].values())
  result = [value < trendline_threshold_filter for value in values]
  mode = find_mode(result)
  perfect_trendline_price_diff_perc[i] = mode


## Applying FIlter

In [17]:
for i in name:
  if perfect_trendline_price_diff_perc[i] and perfect_sr_price_diff_perc[i] < 2:
    print(f'{i} : YES !!')
  else:
    print(f"{i } : NOT MET THE CONDITION")

ADANIPORTS.NS : YES !!
ASIANTILES.NS : NOT MET THE CONDITION
BANDHANBNK.NS : NOT MET THE CONDITION
BHEL.NS : NOT MET THE CONDITION
BODALCHEM.NS : NOT MET THE CONDITION
BRIGADE.NS : NOT MET THE CONDITION
ZENSARTECH.NS : NOT MET THE CONDITION
WELSPUNIND.NS : NOT MET THE CONDITION
VOLTAS.NS : NOT MET THE CONDITION
MANYAVAR.NS : NOT MET THE CONDITION
TATASTEEL.NS : YES !!
SOBHA.NS : NOT MET THE CONDITION
SBICARD.NS : NOT MET THE CONDITION
SAPPHIRE.NS : NOT MET THE CONDITION
SAGCEM.NS : NOT MET THE CONDITION
ROUTE.NS : NOT MET THE CONDITION
RITES.NS : NOT MET THE CONDITION
RBA.NS : NOT MET THE CONDITION
RCOM.NS : NOT MET THE CONDITION
RBLBANK.NS : NOT MET THE CONDITION
RANASUG.NS : NOT MET THE CONDITION
PRESTIGE.NS : NOT MET THE CONDITION
PNCINFRA.NS : YES !!
OFSS.NS : NOT MET THE CONDITION
PAYTM.NS : NOT MET THE CONDITION
NSLNISP.NS : NOT MET THE CONDITION
NMDC.NS : NOT MET THE CONDITION
MTARTECH.NS : NOT MET THE CONDITION
MON100.NS : NOT MET THE CONDITION
MAXHEALTH.NS : NOT MET THE CONDIT